In [ ]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
import numpy as np
from AHRS import AHRS
from quaternions import quaternRotate
from funcions import convertData, convertDataHex, grafica, find_discontinuity, obtenir_x_valors

In [ ]:
dades = pd.read_csv("C:/Users/Usuario/OneDrive/Escritorio/Universitat/TFG/Mataro_postu_IMU/ID_vm54f_DATA_15-03-2023_HORA_17-07-28.351.csv", delimiter=";")
dades

In [ ]:
#print(dades["time"])
convertData(dades,"accelerometerX",10)
convertData(dades,"accelerometerY",10)
convertData(dades,"accelerometerZ",10)

convertDataHex(dades,"gravityVectorX",10)
convertDataHex(dades,"gravityVectorY",10)
convertDataHex(dades,"gravityVectorZ",10)

#convertDataHex(dades,"quaternionW",10)
#convertDataHex(dades,"quaternionX",10)
#convertDataHex(dades,"quaternionY",10)
#convertDataHex(dades,"quaternionZ",10)

convertDataHex(dades,"gyroscopeX",10)
convertDataHex(dades,"gyroscopeY",10)
convertDataHex(dades,"gyroscopeZ",10)

# Exemple carregar dades IMU

In [ ]:
balance_test = dades.loc[dades["test"]==5, ["action","accelerometerX", "accelerometerY", "accelerometerZ", "gravityVectorX", "gravityVectorY", "gravityVectorZ","gyroscopeX", "gyroscopeY", "gyroscopeZ" ]]

llista_ROA = balance_test.index[balance_test["action"] == 1].tolist()
llista_ROC = balance_test.index[balance_test["action"] == 2].tolist()
llista_RGA = balance_test.index[balance_test["action"] == 3].tolist()
llista_RGC = balance_test.index[balance_test["action"] == 4].tolist()
balance_test.drop("action", inplace=True, axis=1)

In [ ]:
index, missing_value = find_discontinuity(llista_ROA)
ROA1 = [*range(llista_ROA[0], missing_value)]

index, missing_value = find_discontinuity(llista_ROC)
ROC1 = [*range(llista_ROC[0], missing_value)]

index, missing_value = find_discontinuity(llista_RGA)
RGA1 = [*range(llista_RGA[0], missing_value)]

index, missing_value = find_discontinuity(llista_RGC)
RGC1 = [*range(llista_RGC[0], missing_value)]

In [ ]:
resultat=[]
resultat=grafica(RGA1)

## Gràfic dades IMU

In [ ]:
x=0
#for i in resultat:
    #i.plot(ax=axes[1,x] ,title="Espuma Ulls tancats")
resultat[0]["gravityVectorX"].plot()
resultat[0]["gravityVectorY"].plot()
resultat[0]["gravityVectorZ"].plot(title="Prova RGA")

plt.show()

In [ ]:
resultat=[]
resultat=grafica(ROA1)

## Trajectòria estimada IMU

In [ ]:
rotated_vector = np.array([1.0,0.0,0.0])
trajectoriax = [rotated_vector[0]]
trajectoriay = [rotated_vector[1]]
trajectoriaz = [rotated_vector[2]]

AHRSalgorithm = AHRS(SamplePeriod=1/40, Kp=1,Ki=1, KpInit=1)

for i in range(len(resultat[0])):
    AHRSalgorithm.Kp = 0
    AHRSalgorithm.UpdateIMU(resultat[0][['gyroscopeX', 'gyroscopeY', 'gyroscopeZ']].iloc[i].values.tolist(), 
                            resultat[0][['accelerometerX', 'accelerometerX', 'accelerometerX']].iloc[i].values.tolist())
    quaternions = AHRSalgorithm.Quaternion
    rotated_vector = quaternRotate(rotated_vector, quaternions)
    #print(rotated_vector)
    trajectoriax.append(trajectoriax[-1] + rotated_vector[0]*deltat)
    trajectoriay.append(trajectoriay[-1] + rotated_vector[1]*deltat)
    trajectoriaz.append(trajectoriaz[-1] + rotated_vector[2]*deltat)

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111, projection = '3d')

ax1.plot(trajectoriax, trajectoriay, trajectoriaz)
ax1.set_title("Desplaçament VM54F ROA")


plt.show()

## Comparació força posturògraf amb forces IMU(Normalitzades)

In [ ]:
if False:
    reader = pd.read_csv("C:/Users/Usuario/OneDrive/Escritorio/Universitat/TFG/Mataro_postu_IMU/2023031504_sve_VM54F.txt", delimiter="\t", encoding="ansi", chunksize=1)
    df_dades = reader.get_chunk()
    #print(df_dades)
    print(df_dades.columns)
    """
    count=0
    for i in df_dades.columns:
        print(i, count)
        count+=1
    """


#reader = pd.read_csv("../../dades/dades posturograf/2022071305_sve_FJ52F.txt", delimiter="\t", encoding="ansi", chunksize=2)
reader = pd.read_csv("C:/Users/Usuario/OneDrive/Escritorio/Universitat/TFG/Mataro_postu_IMU/2023031504_sve_VM54F.txt", delimiter="\t", encoding="ansi", skiprows=1, chunksize=1, header=None)


df_dades = reader.get_chunk()

print(df_dades[17].item())
Val_ROA = df_dades[17].item() #ROA

print(df_dades[22].item())
Val_ROC = df_dades[22].item() #ROC

print(df_dades[27].item())
Val_RGA = df_dades[27].item() #RGA
Val_RGA = 0.0

print(df_dades[32].item())
Val_RGC = df_dades[32].item() #RGC
Val_RGC = 0.0

print(df_dades[53].item())
print(df_dades[54].item())
print(df_dades[55].item())

masa = df_dades[7]
#print(df_dades)
#print(df_dades.columns)
#num_test = df_dades[['Nº_ROA','Nº_ROC','Nº_RGA','Nº_RGC']].values.tolist()

#'Nº_test', 'Val_test',Rep_test', 'EstML_test', 'EstAP_test'
roa = df_dades.iloc[:,16:21].values.tolist()
roc = df_dades.iloc[:,21:26].values.tolist()
rga = df_dades.iloc[:,26:31].values.tolist()
rgc = df_dades.iloc[:,31:36].values.tolist()

##SOM VIS VEST
result = df_dades.iloc[:,53:56].values.tolist()

print(roa)
print(roc)
print(rga)
print(rgc)
print(result)

num_test = roa[0][0] + roc[0][0] + rga[0][0] + rgc[0][0]
print("Total tests")
print(num_test)

In [ ]:
df_dades_postu_FxFy  = pd.read_csv("C:/Users/Usuario/OneDrive/Escritorio/Universitat/TFG/Mataro_postu_IMU/2023031504_sve_VM54F.txt", delimiter="\t", encoding="ansi",skiprows=(17+int(num_test)+1+1200), index_col=False, nrows = 1200)

### Força posturógraf

In [ ]:
df_dades_postu_FxFy.columns

In [ ]:
Test = 'RGC10_'

In [ ]:
Fx_norm = normalize([np.array(df_dades_postu_FxFy[Test + 'Fx'])])
Fy_norm = normalize([np.array(df_dades_postu_FxFy[Test + 'Fy'])])

llista = list(range(0,len(Fx_norm[0])))

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.scatter(llista, Fx_norm[0], s = 1, label="X")
ax1.scatter(llista, Fy_norm[0], s = 1, label="Y")
#ax1.set_title(("Força normalitzada de persona amb Val_ROA = ", Val_ROA))
#ax1.set_title(("Força normalitzada de persona amb Val_ROC = ", Val_ROC))
#ax1.set_title(("Força normalitzada de persona amb Val_RGA = ", Val_RGA))
#ax1.set_title(("Força normalitzada de persona amb Val_RGC = ", Val_RGC))

print("Diferència forces en x: ", max(Fx_norm[0]) - min(Fx_norm[0]))
print("Diferència forces en y: ", max(Fy_norm[0]) - min(Fy_norm[0]))

plt.legend()

plt.show()

### Accelerómetre 

In [ ]:
resultat=[]
resultat=grafica(RGA1)

In [ ]:
Fz_norm_IMU = normalize([resultat[0]["accelerometerZ"]*masa.item()*9.81]) 
Fx_norm_IMU = normalize([resultat[0]["accelerometerX"]*masa.item()*9.81]) 
Fy_norm_IMU = normalize([resultat[0]["accelerometerY"]*masa.item()*9.81]) 

llista = list(range(0,len(Fz_norm_IMU[0])))

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.scatter(llista, Fx_norm_IMU[0], s = 1, label="X")
ax1.scatter(llista, Fy_norm_IMU[0], s = 1, label="Y")
ax1.scatter(llista, Fz_norm_IMU[0], s = 1, label="Z")
#ax1.set_title(("Força normalitzada de persona amb Val_ROA = ", Val_ROA))
ax1.set_title(("Força normalitzada de persona amb Val_ROC = ", Val_ROC))
#ax1.set_title(("Força normalitzada de persona amb Val_RGA = ", Val_RGA))
#ax1.set_title(("Força normalitzada de persona amb Val_RGC = ", Val_RGC))

print("Diferència forces en z: ", max(Fz_norm_IMU[0]) - min(Fz_norm_IMU[0]))
print("Diferència forces en y: ", max(Fy_norm_IMU[0]) - min(Fy_norm_IMU[0]))

plt.legend()

plt.show()

### Vector Gravity

In [ ]:
Fz_norm_IMU_gravity = normalize([resultat[0]["gravityVectorZ"]*masa.item()])
Fx_norm_IMU_gravity = normalize([resultat[0]["gravityVectorX"]*masa.item()])
Fy_norm_IMU_gravity = normalize([resultat[0]["gravityVectorY"]*masa.item()])

llista = list(range(0,len(Fz_norm_IMU_gravity[0])))

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.scatter(llista, Fx_norm_IMU_gravity[0], s = 1, label="X")
ax1.scatter(llista, Fy_norm_IMU_gravity[0], s = 1, label="Y")
ax1.scatter(llista, Fz_norm_IMU_gravity[0], s = 1, label="Z")
#ax1.set_title(("Força normalitzada de persona amb Val_ROA = ", Val_ROA))
ax1.set_title(("Força normalitzada de persona amb Val_ROC = ", Val_ROC))
#ax1.set_title(("Força normalitzada de persona amb Val_RGA = ", Val_RGA))
#ax1.set_title(("Força normalitzada de persona amb Val_RGC = ", Val_RGC))

print("Diferència forces en z: ", max(Fz_norm_IMU_gravity[0]) - min(Fz_norm_IMU_gravity[0]))
print("Diferència forces en y: ", max(Fy_norm_IMU_gravity[0]) - min(Fy_norm_IMU_gravity[0]))

plt.legend()

plt.show()

## Agrupar Dades IMU per edat

Hi ha 5 grups d'edat entre els 68 i els 73 anys, un per cada any.

In [ ]:
dades_forca = pd.read_csv("C:/Users/Usuario/OneDrive/Escritorio/Universitat/TFG/dades_força_IMU.csv", dtype={'Forca_maxima_acc_x': float, 'Forca_maxima_acc_y': float, 'Forca_maxima_grav_x': float, 'Forca_maxima_grav_y': float, 'Puntuacio':float}).drop('Unnamed: 0', axis=1)
dades_forca

In [ ]:
dades_forca_ROA = dades_forca.loc[(dades_forca['Prova'] == 'ROA')]
dades_forca_ROC = dades_forca.loc[(dades_forca['Prova'] == 'ROC')]
dades_forca_RGA = dades_forca.loc[(dades_forca['Prova'] == 'RGA')]
dades_forca_RGC = dades_forca.loc[(dades_forca['Prova'] == 'RGC')]

In [ ]:
dades_forca_ROA_mal = dades_forca_ROA.loc[(dades_forca_ROA['Puntuacio'] < 90)]
dades_forca_ROA_bo = dades_forca_ROA.loc[(dades_forca_ROA['Puntuacio']) >= 90]

dades_forca_ROC_mal = dades_forca_ROC.loc[(dades_forca_ROC['Puntuacio'] < 90)]
dades_forca_ROC_bo = dades_forca_ROC.loc[(dades_forca_ROC['Puntuacio']) >= 90]

dades_forca_RGA_mal = dades_forca_RGA.loc[(dades_forca_RGA['Puntuacio'] < 90)]
dades_forca_RGA_bo = dades_forca_RGA.loc[(dades_forca_RGA['Puntuacio']) >= 90]

dades_forca_RGC_mal = dades_forca_RGC.loc[(dades_forca_RGC['Puntuacio'] < 80)]
dades_forca_RGC_bo = dades_forca_RGC.loc[(dades_forca_RGC['Puntuacio']) >= 80]

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.scatter(dades_forca_ROA_bo['Forca_maxima_acc_x'], dades_forca_ROA_bo['Forca_maxima_acc_y'], s = 1, label="Pacients amb valoració >= 90")
ax1.scatter(dades_forca_ROA_mal['Forca_maxima_acc_x'], dades_forca_ROA_mal['Forca_maxima_acc_y'], s = 1, label="Pacients amb valoració < 90")
ax1.set_title("Màxima acceleració a la prova ROA")
plt.legend()


plt.show()

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.scatter(dades_forca_ROC_bo['Forca_maxima_acc_x'], dades_forca_ROC_bo['Forca_maxima_acc_y'], s = 1, label="Pacients amb valoració >= 90")
ax1.scatter(dades_forca_ROC_mal['Forca_maxima_acc_x'], dades_forca_ROC_mal['Forca_maxima_acc_y'], s = 1, label="Pacients amb valoració < 90")
ax1.set_title("Màxima acceleració a la prova ROC")
plt.legend()


plt.show()

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.scatter(dades_forca_RGA_bo['Forca_maxima_acc_x'], dades_forca_RGA_bo['Forca_maxima_acc_y'], s = 1, label="Pacients amb valoració >= 90")
ax1.scatter(dades_forca_RGA_mal['Forca_maxima_acc_x'], dades_forca_RGA_mal['Forca_maxima_acc_y'], s = 1, label="Pacients amb valoració < 90")
ax1.set_title("Màxima acceleració a la prova RGA")
plt.legend()


plt.show()

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.scatter(dades_forca_RGC_bo['Forca_maxima_acc_x'], dades_forca_RGC_bo['Forca_maxima_acc_y'], s = 1, label="Pacients amb valoració >= 90")
ax1.scatter(dades_forca_RGC_mal['Forca_maxima_acc_x'], dades_forca_RGC_mal['Forca_maxima_acc_y'], s = 1, label="Pacients amb valoració < 90")
ax1.set_title("Màxima acceleració a la prova RGC")
plt.legend()


plt.show()

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.scatter(dades_forca_ROA_bo['Forca_maxima_grav_x'], dades_forca_ROA_bo['Forca_maxima_grav_y'], s = 1, label="Pacients amb valoració >= 90")
ax1.scatter(dades_forca_ROA_mal['Forca_maxima_grav_x'], dades_forca_ROA_mal['Forca_maxima_grav_y'], s = 1, label="Pacients amb valoració < 90")
ax1.set_title("Màxim vector gravetat a la prova ROA")
ax1.set_xlim(-0.001,0.25)
ax1.set_ylim(-0.001,0.25)
plt.legend()


plt.show()

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.scatter(dades_forca_ROC_bo['Forca_maxima_grav_x'], dades_forca_ROC_bo['Forca_maxima_grav_y'], s = 1, label="Pacients amb valoració >= 90")
ax1.scatter(dades_forca_ROC_mal['Forca_maxima_grav_x'], dades_forca_ROC_mal['Forca_maxima_grav_y'], s = 1, label="Pacients amb valoració < 90")
ax1.set_title("Màxim vector gravetat a la prova ROC")
plt.legend()


plt.show()

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.scatter(dades_forca_RGA_bo['Forca_maxima_grav_x'], dades_forca_RGA_bo['Forca_maxima_grav_y'], s = 1, label="Pacients amb valoració >= 90")
ax1.scatter(dades_forca_RGA_mal['Forca_maxima_grav_x'], dades_forca_RGA_mal['Forca_maxima_grav_y'], s = 1, label="Pacients amb valoració < 90")
ax1.set_title("Màxim vector gravetat a la prova RGA")
plt.legend()


plt.show()

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.scatter(dades_forca_RGC_bo['Forca_maxima_grav_x'], dades_forca_RGC_bo['Forca_maxima_grav_y'], s = 1, label="Pacients amb valoració >= 90")
ax1.scatter(dades_forca_RGC_mal['Forca_maxima_grav_x'], dades_forca_RGC_mal['Forca_maxima_grav_y'], s = 1, label="Pacients amb valoració < 90")
ax1.set_title("Màxim vector gravetat a la prova RGC")
ax1.set_xlim(-0.001,0.25)
ax1.set_ylim(-0.001,0.25)
plt.legend()


plt.show()

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.scatter(dades_forca_ROA_bo['Desplacament_maxim_x'], dades_forca_ROA_bo['Desplacament_maxim_y'], s = 1, label="Pacients amb valoració >= 90")
ax1.scatter(dades_forca_ROA_mal['Desplacament_maxim_x'], dades_forca_ROA_mal['Desplacament_maxim_y'], s = 1, label="Pacients amb valoració < 90")
ax1.set_title("Màxim desplaçament a la prova ROA")
plt.legend()


plt.show()

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.scatter(dades_forca_ROC_bo['Desplacament_maxim_x'], dades_forca_ROC_bo['Desplacament_maxim_y'], s = 1, label="Pacients amb valoració >= 90")
ax1.scatter(dades_forca_ROC_mal['Desplacament_maxim_x'], dades_forca_ROC_mal['Desplacament_maxim_y'], s = 1, label="Pacients amb valoració < 90")
ax1.set_title("Màxim desplaçament a la prova ROC")
plt.legend()


plt.show()

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.scatter(dades_forca_RGA_bo['Desplacament_maxim_x'], dades_forca_RGA_bo['Desplacament_maxim_y'], s = 1, label="Pacients amb valoració >= 90")
ax1.scatter(dades_forca_RGA_mal['Desplacament_maxim_x'], dades_forca_RGA_mal['Desplacament_maxim_y'], s = 1, label="Pacients amb valoració < 90")
ax1.set_title("Màxim desplaçament a la prova RGA")
plt.legend()


plt.show()

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.scatter(dades_forca_RGC_bo['Desplacament_maxim_x'], dades_forca_RGC_bo['Desplacament_maxim_y'], s = 1, label="Pacients amb valoració >= 90")
ax1.scatter(dades_forca_RGC_mal['Desplacament_maxim_x'], dades_forca_RGC_mal['Desplacament_maxim_y'], s = 1, label="Pacients amb valoració < 90")
ax1.set_title("Màxim desplaçament a la prova RGC")
plt.legend()


plt.show()